Name : Pratik Warade

# Assignment 4: Negamax with Alpha-Beta Pruning and Iterative Deepening

# Table of Contents
* [Assignment 4: Negamax with Alpha-Beta Pruning and Iterative Deepening](#Assignment-4:-Negamax-with-Alpha-Beta-Pruning-and-Iterative-Deepening)
	* [Initial Code](#Initial-Code)
	* [negamaxIDS](#negamaxIDS)
	* [negamaxIDSab](#negamaxIDSab)
	* [Grading](#Grading)
	* [Extra Credit](#Extra-Credit)


In this assignment, I  investigated the advantages of alpha-beta
pruning applied to Tic-Tac-Toe. 

## Initial Code 

In [1]:
import random

In [2]:
def negamax(game, depthLeft):
    # If at terminal state or depth limit, return utility value and move None
    if game.isOver() or depthLeft == 0:
        return game.getUtility(), None # call to negamax knows the move
    # Find best move and its value from current state
    bestValue, bestMove = None, None
    for move in game.getMoves():
        # Apply a move to current state
        game.makeMove(move)
        # Use depth-first search to find eventual utility value and back it up.
        #  Negate it because it will come back in context of next player
        value, _ = negamax(game, depthLeft-1)
        # Remove the move from current state, to prepare for trying a different move
        game.unmakeMove(move)
        if value is None:
            continue
        value = - value
        if bestValue is None or value > bestValue:
            # Value for this move is better than moves tried so far from this state.
            bestValue, bestMove = value, move
    return bestValue, bestMove

In [3]:
class TTT(object):

    def __init__(self):
        self.board = [' ']*9
        self.player = 'X'
        self.movesExplored =0
        if False:
            self.board = ['X', 'X', ' ', 'X', 'O', 'O', ' ', ' ', ' ']
            self.player = 'O'
        self.playerLookAHead = self.player

    def locations(self, c):
        return [i for i, mark in enumerate(self.board) if mark == c]

    def getMoves(self):
        moves = self.locations(' ')
        return moves

    def getUtility(self):
        whereX = self.locations('X')
        whereO = self.locations('O')
        wins = [[0, 1, 2], [3, 4, 5], [6, 7, 8],
                [0, 3, 6], [1, 4, 7], [2, 5, 8],
                [0, 4, 8], [2, 4, 6]]
        isXWon = any([all([wi in whereX for wi in w]) for w in wins])
        isOWon = any([all([wi in whereO for wi in w]) for w in wins])
        if isXWon:
            return 1 if self.playerLookAHead is 'X' else -1
        elif isOWon:
            return 1 if self.playerLookAHead is 'O' else -1
        elif ' ' not in self.board:
            return 0
        else:
            return None  

    def isOver(self):
        return self.getUtility() is not None
    
    def getNumberMovesExplored(self):
        return self.movesExplored
    def getWinningValue(self):
        return(1)

    def makeMove(self, move):
        self.board[move] = self.playerLookAHead
        self.playerLookAHead = 'X' if self.playerLookAHead == 'O' else 'O'
        self.movesExplored =self.movesExplored +1

    def changePlayer(self):
        self.player = 'X' if self.player == 'O' else 'O'
        self.playerLookAHead = self.player

    def unmakeMove(self, move):
        self.board[move] = ' '
        self.playerLookAHead = 'X' if self.playerLookAHead == 'O' else 'O'

    def __str__(self):
        s = '{}|{}|{}\n-----\n{}|{}|{}\n-----\n{}|{}|{}'.format(*self.board)
        return s

In [4]:
def opponent(board):
    return board.index(' ')
def playGame(game,opponent,depthLimit):
    print(game)
    while not game.isOver():
        score,move = negamax(game,depthLimit)
        if move == None :
            print('move is None. Stopping.')
            break
        game.makeMove(move)
        print('Player', game.player, 'to', move, 'for score' ,score)
        print(game)
        if not game.isOver():
            game.changePlayer()
            opponentMove = opponent(game.board)
            game.makeMove(opponentMove)
            print('Player', game.player, 'to', opponentMove)  
            print(game)
            game.changePlayer()

### Effective Branching Factor
One way to characterize the quality or efficiency of an heuristic is the effective branching factor b`*`.The quality of the heuristic mostly depends on the ‘effective branching factor’ value – Following is the definition for effective branching factor. If the total number of nodes generated by A* search for a problem space is N, and the solution tree depth is d, then the effective branching factor b is defined using the following equation
    N + 1 = 1 + b + b2 + b3 + … … … … … … . + b^ d
    
    which can be written as :  N = ( 1- b^(d+1))/(1-b)
    
Using this an approximate solution for b can be obtained mathematically, which is called the effective branching factor.

In [5]:
def ebfpoly(b,depth):
    #calculate N
    power=b**(depth+1)
    if b==1:  
        evala=1
    else:
        evala=(1-power)/(1-b)
    return evala    
    
def ebf(nNodes,depth,precision=0.001):
    #check weather depth is greather than number of nodes.
    if(depth>nNodes):
        fail= print("Depth Cannot be larger than number of Node..Retry again ")
        return fail
    #set binary tree upper=nNodes and lower bound=0
    binaryTree_low=0.0
    binaryTree_high=nNodes   
    cnt=0
    #calculate value of N from tmp b value. Shown in above cell
    #call ebfpoly to get value of solved equation
    eva=ebfpoly(binaryTree_low,depth)
    while abs(eva-nNodes)>precision and cnt<10000:    #check precision with precison of calculated value from ploynomial function 
        midpoint=(binaryTree_low + binaryTree_high)/2
        eva=ebfpoly(midpoint,depth)
        if(eva-nNodes)<0: #check in which range caluated value fall if less change low to mid else high =mid
            binaryTree_low=midpoint
        else:
            binaryTree_high=midpoint     
        cnt=cnt+1
    return binaryTree_high


# This function take object game and calculates number of moves explored, Number of 'X' and Number of moved tiles
#Once we get all required value , ebf fuction is called to get ebf value
def cal_ebf(game):
    No_moves=game.getNumberMovesExplored()
    x=game.board.count('X')
    o=game.board.count('O')
    tiles_Moved=x+o
    ans=ebf(No_moves,tiles_Moved)
    return [x,No_moves,tiles_Moved,ans]

## negamaxIDS 



Write a new function named `negamaxIDS` that performs an iterative deepening negamax search.  Replace the first line in the `while` loop of `playGame` with

        score,move = negamaxIDS(game,depthLimit)
        
where `depthLimit` is now the maximum depth and multiple `negamax` searches are performed for depth limits of $1, 2, \ldots,$ maximum depth.

But, when should you stop?  Can you stop before readhing the depthLimit?  If not, there is no point to doing iterative deepening.

For Tic-Tac-Toe, we can stop as soon as a call to `negamax` returns a winning move.  This will have a value of 1 for Tic-Tac-Toe.  To keep our `negamaxIDS` function general, add a method called `getWinningValue` to the `TTT` class that just returns 1.  Then `negamaxIDS` can call `game.getWinningValue()` to determine the value of a winning move for this game.  If the maximum depth is reached and no winning move has been found, return the best move found over all depth limts.

In [6]:
def opponent(board):
    return board.index(' ')

def playGameIDS(game,opponent,depthLimit):
    print(game)
    while not game.isOver():
        score,move = negamaxIDS(game,depthLimit)
        if move == None :
            print('move is None. Stopping.')
            break
        game.makeMove(move)
        print('Player', game.player, 'to', move, 'for score' ,score)
        print(game)
        if not game.isOver():
            game.changePlayer()
            opponentMove = opponent(game.board)
            game.makeMove(opponentMove)
            print('Player', game.player, 'to', opponentMove)   
            print(game)
            game.changePlayer()

In [7]:
def negamaxIDS(game,maxDepth):
    if game.isOver() or maxDepth == 0:
        return game.getUtility(), None
    # Find best move and its value from current state
    bestValue = -float('infinity')
    bestMove = None
    for depth in range(maxDepth):
        #rest game to empty
      #  game.__init__()
      #  print("XXXXPPPPP \n",game)
       # print('\n')
        value,move=negamax(game,depth)           
        if value is None:
            continue
        #value = - value
        # Remove the move from current state, to prepare for trying a different move
       # game.unmakeMove(move)
        if  value > bestValue and value!= float('infinity'):
            # Value for this move is better than moves tried so far from this state.
            bestValue, bestMove = value, move
        if bestValue==game.getWinningValue():
            return bestValue,bestMove
         
   
    return bestValue,bestMove
        

## negamaxIDSab

In [8]:
def opponent(board):
    return board.index(' ')

def playGameIDSab(game,opponent,depthLimit):
    #game=TTT()
    print(game)
    
    while not game.isOver():
        score,move = negamaxIDSab(game,depthLimit)
        if move == None :
            print('move is None. Stopping.')
            break
        game.makeMove(move)
        print('Player', game.player, 'to', move, 'for score' ,score)
        print(game)
        if not game.isOver():
            game.changePlayer()
            opponentMove = opponent(game.board)
            game.makeMove(opponentMove)
            print('Player', game.player, 'to', opponentMove)
            print(game)
            game.changePlayer()
        

In [9]:
def negamax_ab(game, depthLeft,alpha,beta):
   # alpha = -float('inf')
   # beta = float("inf")
    # If at terminal state or depth limit, return utility value and move None
    if game.isOver() or depthLeft == 0:
        return game.getUtility(), None # call to negamax knows the move
    # Find best move and its value from current state
    bestValue, bestMove = None, None
    for move in game.getMoves():
      #  alpha , beta = - beta,-alpha
        # Apply a move to current state
        game.makeMove(move)
        # Use depth-first search to find eventual utility value and back it up.
        #  Negate it because it will come back in context of next player
        value, _ = negamax_ab(game, depthLeft-1,-beta,-alpha)
        # Remove the move from current state, to prepare for trying a different move
        game.unmakeMove(move)
        if value is None:
            continue
        value = - value
        if bestValue is None or value > bestValue:
            # Value for this move is better than moves tried so far from this state.
            bestValue, bestMove = value, move
        if(value>alpha):
                alpha=value
        if bestValue >= beta:
            return bestValue, bestMove
    return bestValue, bestMove

In [10]:
def negamaxIDSab(game,maxDepth):
    #ab=True
    alpha = -float('inf')
    beta = float("inf")
    if game.isOver() or maxDepth == 0:
        return game.getUtility(), None
    # Find best move and its value from current state
    bestValue = -float('infinity')
    bestMove = None
    for depth in range(maxDepth):
        #****swap and negate alpha, beta
        value,move=negamax_ab(game,depth,alpha,beta)
       # print("TEST1",game.getWinningValue())
        #print("TEST2",value,move)      
         
        if value is None:
            continue
        #value = - value
        # Remove the move from current state, to prepare for trying a different move
       # game.unmakeMove(move)
        if bestValue is None or value > bestValue:
            # Value for this move is better than moves tried so far from this state.
            bestValue, bestMove = value, move
        if bestValue==game.getWinningValue():
            return bestValue,bestMove
         
   
    return bestValue,bestMove
    

## playGames

Now duplicating the game playing loop so three complete tic-tac-toe games are played.   For the first game, I used `negamax`. For the second game,  `negamaxIDS` is used.  For the third game,  `negamaxIDSab ` is used.  At the end of each game, I printed the number of X's in the final board, the number moves explored, the depth of the game which is the number of moves made by X and O, and the effective branching factor.  When you run `playGames` you will see the tic-tac-toe positions after each move and, after all games are done, a line for each game with details are diplayed.
   


In [11]:
#playgame make TTT object and passed it required alogrithm which then call cal_ebf function to calcluate ebf and print it
def playGames(opponent, depth):
    print("negamax:")
    game = TTT()
    playGame(game,opponent,depth)
    
    
    print('\n')
    print("negamaxIDS:")
    game_IDS = TTT()
    playGameIDS(game_IDS,opponent,depth)
    
    print('\n')
    print("negamaxIDSab:")
    gameIDSab = TTT()
    playGameIDSab(gameIDSab,opponent,depth)
    
    print('\n')
    ans=cal_ebf(game)
    print("negamax made  "+ str(ans[0] )+"  moves. "+ str(ans[1]) +" moves explored for ebf("+str(ans[1])+", "+str(ans[2])+") of "+str(round(ans[3],2)))
    
    ans_ids=cal_ebf(game_IDS)
    print("negamaxIDS made  "+ str(ans_ids[0] )+"  moves. "+ str(ans_ids[1]) +" moves explored for ebf("+str(ans_ids[1])+", "+str(ans_ids[2])+") of "+str(round(ans_ids[3],2)))
    
    
    ans_idsab=cal_ebf(gameIDSab)
    print("negamaxIDSab made  "+ str(ans_idsab[0] )+"  moves. "+ str(ans_idsab[1]) +" moves explored for ebf("+str(ans_idsab[1])+", "+str(ans_idsab[2])+") of "+str(round(ans_idsab[3],2)))
    

In [12]:
playGames(opponent, 10)

negamax:
 | | 
-----
 | | 
-----
 | | 
Player X to 0 for score 0
X| | 
-----
 | | 
-----
 | | 
Player O to 1
X|O| 
-----
 | | 
-----
 | | 
Player X to 3 for score 1
X|O| 
-----
X| | 
-----
 | | 
Player O to 2
X|O|O
-----
X| | 
-----
 | | 
Player X to 4 for score 1
X|O|O
-----
X|X| 
-----
 | | 
Player O to 5
X|O|O
-----
X|X|O
-----
 | | 
Player X to 6 for score 1
X|O|O
-----
X|X|O
-----
X| | 


negamaxIDS:
 | | 
-----
 | | 
-----
 | | 
Player X to 0 for score 1
X| | 
-----
 | | 
-----
 | | 
Player O to 1
X|O| 
-----
 | | 
-----
 | | 
Player X to 3 for score 1
X|O| 
-----
X| | 
-----
 | | 
Player O to 2
X|O|O
-----
X| | 
-----
 | | 
Player X to 6 for score 1
X|O|O
-----
X| | 
-----
X| | 


negamaxIDSab:
 | | 
-----
 | | 
-----
 | | 
Player X to 0 for score 1
X| | 
-----
 | | 
-----
 | | 
Player O to 1
X|O| 
-----
 | | 
-----
 | | 
Player X to 3 for score 1
X|O| 
-----
X| | 
-----
 | | 
Player O to 2
X|O|O
-----
X| | 
-----
 | | 
Player X to 6 for score 1
X|O|O
-----
X| | 
-----
X| | 


n

## Grading

In [13]:
%run -i A4grader.py


Testing negamax starting from ['O', 'X', ' ', 'O', ' ', ' ', ' ', 'X', ' ']

--- 10/10 points. negamax correctly returns value of 1

--- 10/10 points. negamax correctly explored 124 states.

Testing negamax starting from ['O', 'X', 'X', 'O', 'O', ' ', ' ', 'X', ' ']

--- 10/10 points. negamax correctly returns value of -1 and move of 5

Testing negamaxIDS with max depth of 5, starting from ['O', 'X', 'X', 'O', 'O', ' ', ' ', 'X', ' ']

--- 10/10 points. negamaxIDS correctly returns value of -1 and move of 5

Testing negamaxIDSab starting from ['O', 'X', 'X', 'O', 'O', ' ', ' ', 'X', ' ']

--- 20/20 points. negamaxIDSab correctly returns value of -1 and move of 5

Testing playGame with opponent that always plays in highest numbered position.
 | | 
-----
 | | 
-----
 | | 
Player X to 0 for score 0
X| | 
-----
 | | 
-----
 | | 
Player O to 8
X| | 
-----
 | | 
-----
 | |O
Player X to 2 for score 1
X| |X
-----
 | | 
-----
 | |O
Player O to 7
X| |X
-----
 | | 
-----
 |O|O
Player X to 1 for 

## Extra Credit 

Earn one extra credit point for each of the following.

  - Implement another game and repeat the above steps.

  - Implement a random move chooser as the opponent (Player O) and determine how many times Player X can win against this opponent as an average over multiple games.

#### Extra Credit : Random Move Chooser

In [14]:
# For 1st extra credit , I used random generation of index for appoenent i.e for 'O'. In this code, I search through all blank spaces and store indexes.
# Once, we get all index in list, I randomly selects any number and return that as index

def opponent_random(board):
    tmp=[]
    for val in range(0, 9):
        if ' ' in board[val]:
            tmp.append(val)
    index=random.choice(tmp)
    return index
def playGame_random(game,opponent,depthLimit):
    print(game)
    while not game.isOver():
        score,move = negamax(game,depthLimit)
        if move == None :
            print('move is None. Stopping.')
            break
        game.makeMove(move)
        print('Player', game.player, 'to', move, 'for score' ,score)
        print(game)
        if not game.isOver():
            game.changePlayer()
            opponentMove = opponent_random(game.board)
            game.makeMove(opponentMove)
            print('Player', game.player, 'to', opponentMove)   
            print(game)
            game.changePlayer()
            
def playGameIDS_random(game,opponent,depthLimit):
    print(game)
    while not game.isOver():
        score,move = negamaxIDS(game,depthLimit)
        if move == None :
            print('move is None. Stopping.')
            break
        game.makeMove(move)
        print('Player', game.player, 'to', move, 'for score' ,score)
        print(game)
        if not game.isOver():
            game.changePlayer()
            opponentMove = opponent_random(game.board)
            game.makeMove(opponentMove)
            print('Player', game.player, 'to', opponentMove)  
            print(game)
            game.changePlayer()
            
def playGameIDSab_random(game,opponent,depthLimit):
    #game=TTT()
    print(game)
    
    while not game.isOver():
        score,move = negamaxIDSab(game,depthLimit)
        if move == None :
            print('move is None. Stopping.')
            break
        game.makeMove(move)
        print('Player', game.player, 'to', move, 'for score' ,score)
        print(game)
        if not game.isOver():
            game.changePlayer()
            opponentMove = opponent_random(game.board)
            game.makeMove(opponentMove)
            print('Player', game.player, 'to', opponentMove)
            print(game)
            game.changePlayer()
        

In [15]:
#playgame make TTT object and passed it required alogrithm which then call cal_ebf function to calcluate ebf and print it
def playGames_random(opponent, depth):
    print("negamax:")
    game = TTT()
    playGame_random(game,opponent,depth)
    
    
    print('\n')
    print("negamaxIDS:")
    game_IDS = TTT()
    playGameIDS_random(game_IDS,opponent,depth)
    
    print('\n')
    print("negamaxIDSab:")
    gameIDSab = TTT()
    playGameIDSab_random(gameIDSab,opponent,depth)
    
    print('\n')
    ans=cal_ebf(game)
    print("negamax made  "+ str(ans[0] )+"  moves. "+ str(ans[1]) +" moves explored for ebf("+str(ans[1])+", "+str(ans[2])+") of "+str(round(ans[3],2)))
    
    ans_ids=cal_ebf(game_IDS)
    print("negamaxIDS made  "+ str(ans_ids[0] )+"  moves. "+ str(ans_ids[1]) +" moves explored for ebf("+str(ans_ids[1])+", "+str(ans_ids[2])+") of "+str(round(ans_ids[3],2)))
    
    
    ans_idsab=cal_ebf(gameIDSab)
    print("negamaxIDSab made  "+ str(ans_idsab[0] )+"  moves. "+ str(ans_idsab[1]) +" moves explored for ebf("+str(ans_idsab[1])+", "+str(ans_idsab[2])+") of "+str(round(ans_idsab[3],2)))
    

In [16]:
playGames_random(opponent, 10)

negamax:
 | | 
-----
 | | 
-----
 | | 
Player X to 0 for score 0
X| | 
-----
 | | 
-----
 | | 
Player O to 6
X| | 
-----
 | | 
-----
O| | 
Player X to 1 for score 1
X|X| 
-----
 | | 
-----
O| | 
Player O to 3
X|X| 
-----
O| | 
-----
O| | 
Player X to 2 for score 1
X|X|X
-----
O| | 
-----
O| | 


negamaxIDS:
 | | 
-----
 | | 
-----
 | | 
Player X to 0 for score 1
X| | 
-----
 | | 
-----
 | | 
Player O to 6
X| | 
-----
 | | 
-----
O| | 
Player X to 1 for score 1
X|X| 
-----
 | | 
-----
O| | 
Player O to 3
X|X| 
-----
O| | 
-----
O| | 
Player X to 2 for score 1
X|X|X
-----
O| | 
-----
O| | 


negamaxIDSab:
 | | 
-----
 | | 
-----
 | | 
Player X to 0 for score 1
X| | 
-----
 | | 
-----
 | | 
Player O to 5
X| | 
-----
 | |O
-----
 | | 
Player X to 1 for score 1
X|X| 
-----
 | |O
-----
 | | 
Player O to 7
X|X| 
-----
 | |O
-----
 |O| 
Player X to 2 for score 1
X|X|X
-----
 | |O
-----
 |O| 


negamax made  3  moves. 557678 moves explored for ebf(557678, 5) of 13.89
negamaxIDS made  3  moves. 

In [17]:
playGames_random(opponent, 10)

negamax:
 | | 
-----
 | | 
-----
 | | 
Player X to 0 for score 0
X| | 
-----
 | | 
-----
 | | 
Player O to 8
X| | 
-----
 | | 
-----
 | |O
Player X to 2 for score 1
X| |X
-----
 | | 
-----
 | |O
Player O to 5
X| |X
-----
 | |O
-----
 | |O
Player X to 1 for score 1
X|X|X
-----
 | |O
-----
 | |O


negamaxIDS:
 | | 
-----
 | | 
-----
 | | 
Player X to 0 for score 1
X| | 
-----
 | | 
-----
 | | 
Player O to 2
X| |O
-----
 | | 
-----
 | | 
Player X to 3 for score 1
X| |O
-----
X| | 
-----
 | | 
Player O to 6
X| |O
-----
X| | 
-----
O| | 
Player X to 4 for score 1
X| |O
-----
X|X| 
-----
O| | 
Player O to 7
X| |O
-----
X|X| 
-----
O|O| 
Player X to 5 for score 1
X| |O
-----
X|X|X
-----
O|O| 


negamaxIDSab:
 | | 
-----
 | | 
-----
 | | 
Player X to 0 for score 1
X| | 
-----
 | | 
-----
 | | 
Player O to 7
X| | 
-----
 | | 
-----
 |O| 
Player X to 1 for score 1
X|X| 
-----
 | | 
-----
 |O| 
Player O to 3
X|X| 
-----
O| | 
-----
 |O| 
Player X to 2 for score 1
X|X|X
-----
O| | 
-----
 |O| 


n

In [18]:
playGames_random(opponent, 10)

negamax:
 | | 
-----
 | | 
-----
 | | 
Player X to 0 for score 0
X| | 
-----
 | | 
-----
 | | 
Player O to 7
X| | 
-----
 | | 
-----
 |O| 
Player X to 2 for score 1
X| |X
-----
 | | 
-----
 |O| 
Player O to 3
X| |X
-----
O| | 
-----
 |O| 
Player X to 1 for score 1
X|X|X
-----
O| | 
-----
 |O| 


negamaxIDS:
 | | 
-----
 | | 
-----
 | | 
Player X to 0 for score 1
X| | 
-----
 | | 
-----
 | | 
Player O to 3
X| | 
-----
O| | 
-----
 | | 
Player X to 1 for score 1
X|X| 
-----
O| | 
-----
 | | 
Player O to 8
X|X| 
-----
O| | 
-----
 | |O
Player X to 2 for score 1
X|X|X
-----
O| | 
-----
 | |O


negamaxIDSab:
 | | 
-----
 | | 
-----
 | | 
Player X to 0 for score 1
X| | 
-----
 | | 
-----
 | | 
Player O to 6
X| | 
-----
 | | 
-----
O| | 
Player X to 1 for score 1
X|X| 
-----
 | | 
-----
O| | 
Player O to 7
X|X| 
-----
 | | 
-----
O|O| 
Player X to 2 for score 1
X|X|X
-----
 | | 
-----
O|O| 


negamax made  3  moves. 556454 moves explored for ebf(556454, 5) of 13.89
negamaxIDS made  3  moves. 

For random opponent, I ran palygame 3 times which ran  3 algrothims on each of it. 
* Determine how many times Player X can win against this opponent as an average over multiple games.  
Ans:   
```In each of the games, Opponent 'O' never wins. As,   'X' is made an intelligent player using negamx, negamaxIds, and negamIdsab. 'X' wins : 9 |  'O' wins : 0 . Hence, Average is 9 of 'X'.```

#### Extra Credit  Game 

The Game I have implmented bellow is called as ```"Treblecross"``` .  Treblecross is a degenerate Tic-tac toe variant. The game is an octal game, played on a one-dimensional board and both players play using the same piece(an X or a black chip). Each player on their turn plays a piece in an unoccupied space. The game is won if a player on their turn creates a line of three pieces(X's or black chips) in a row.            ... Reference (Wikipedia)
<img src="https://upload.wikimedia.org/wikipedia/commons/8/82/Treblecross.png">

As in above game, there is only one notation for both player. I have given representation as 'X' to 1st player and  to second player. I tried '-X' but it was getting hard to look 3 consecutive  win which may include 'X' and '-X'. And as per game instruction it should be 'X'. Also, this game can be played for n number of board horizontal board size, I choose 11 which is just a random guess.

In [19]:
class TrebleCross(object):

    def __init__(self):
        self.board = [' ']*11
        self.player = 'X'
        self.movesExplored =0
        if False:
            self.board = ['X', 'X', ' ', 'X', 'X', 'X', ' ', ' ', ' ',' ']
            self.player = 'X'
        self.playerLookAHead = self.player

    def locations(self, c):
        return [i for i, mark in enumerate(self.board) if mark == c]

    def getMoves(self):
        moves = self.locations(' ')
        return moves

    def getUtility(self):
        whereX = self.locations('X')
        #where_X = self.locations('X')
        wins = [[0, 1, 2], [1, 2, 3], [2, 3, 4],
                [3, 4, 5], [4, 5, 6], [5, 6, 7],
                [6,7,8],[7,8,9],
                [2,3,4],[3,4,5],
                [4,5,6],[5,6,7],[6,7,8],[7,8,9],
                [8,9,10]
                
                
                ]
        isXWon = any([all([wi in whereX for wi in w]) for w in wins])
        if isXWon:
            return 1 if self.playerLookAHead is 'X' else -1
        elif ' ' not in self.board:
            return 0
        else:
            return None 

    def isOver(self):
        return self.getUtility() is not None
    
    def getNumberMovesExplored(self):
        return self.movesExplored
    def getWinningValue(self):
        return(1)

    def makeMove(self, move):
        self.board[move] = self.playerLookAHead
        self.playerLookAHead = 'X' 
        self.movesExplored =self.movesExplored +1

    def changePlayer(self):
        self.player = 'X' 
        self.playerLookAHead = self.player

    def unmakeMove(self, move):
        self.board[move] = ' '
        self.playerLookAHead = 'X' 

    def __str__(self):
        s = '|{}|{}|{}|{}|{}|{}|{}|{}|{}|{}|{}|'.format(*self.board)
        return s

In [20]:
def opponent_random_treble(board):
    tmp=[]
    for val in range(0, 11):
        if ' ' in board[val]:
            tmp.append(val)
    index=random.choice(tmp)
    return index
def playGame_tre(game,opponent_random_treble,depthLimit):
    print(game)
    
    while not game.isOver():
        score,move = negamax(game,depthLimit)
        if move == None :
            print('move is None. Stopping.')
            break
        game.makeMove(move)
        print('Player', game.player, 'to', move, 'for score' ,score)
        
        print(game)
        if not game.isOver():
            game.changePlayer()
            opponentMove = opponent_random_treble(game.board)
            game.makeMove(opponentMove)
            print('Player', game.player, 'to', opponentMove)   
            print(game)
            game.changePlayer()
            
            
            
def playGame_treIDS(game,opponent,depthLimit):
    print(game)
    while not game.isOver():
        score,move = negamaxIDS(game,depthLimit)
        if move == None :
            print('move is None. Stopping.')
            break
        game.makeMove(move)
        print('Player', game.player, 'to', move, 'for score' ,score)
        print(game)
        if not game.isOver():
            game.changePlayer()
            opponentMove = opponent_random_treble(game.board)
            game.makeMove(opponentMove)
            print('Player', game.player, 'to', opponentMove)  
            print(game)
            game.changePlayer()
            
def playGame_treab(game,opponent,depthLimit):
    #game=TTT()
    print(game)
    
    while not game.isOver():
        score,move = negamaxIDSab(game,depthLimit)
        if move == None :
            print('move is None. Stopping.')
            break
        game.makeMove(move)
        print('Player', game.player, 'to', move, 'for score' ,score)
        print(game)
        if not game.isOver():
            game.changePlayer()
            opponentMove = opponent_random_treble(game.board)
            game.makeMove(opponentMove)
            print('Player', game.player, 'to', opponentMove)
            print(game)
            game.changePlayer()
        

In [21]:
#playgame make TTT object and passed it required alogrithm which then call cal_ebf function to calcluate ebf and print it
def playGames_random_extra_credit(opponent, depth):
    print("negamax:")
    game = TrebleCross()
    playGame_tre(game,opponent,depth)
    
    
    print('\n')
    print("negamaxIDS:")
    game_IDS = TrebleCross()
    playGame_treIDS(game_IDS,opponent,depth)
    
    print('\n')
    print("negamaxIDSab:")
    gameIDSab = TrebleCross()
    playGame_treab(gameIDSab,opponent,depth)
    
    print('\n')
    ans=cal_ebf(game)
    print("negamax made  "+ str(ans[0] )+"  moves. "+ str(ans[1]) +" moves explored for ebf("+str(ans[1])+", "+str(ans[2])+") of "+str(round(ans[3],2)))
    
    ans_ids=cal_ebf(game_IDS)
    print("negamaxIDS made  "+ str(ans_ids[0] )+"  moves. "+ str(ans_ids[1]) +" moves explored for ebf("+str(ans_ids[1])+", "+str(ans_ids[2])+") of "+str(round(ans_ids[3],2)))
    
    
    ans_idsab=cal_ebf(gameIDSab)
    print("negamaxIDSab made  "+ str(ans_idsab[0] )+"  moves. "+ str(ans_idsab[1]) +" moves explored for ebf("+str(ans_idsab[1])+", "+str(ans_idsab[2])+") of "+str(round(ans_idsab[3],2)))
    

In [22]:
playGames_random_extra_credit(opponent, 10)

negamax:
| | | | | | | | | | | |
Player X to 0 for score 1
|X| | | | | | | | | | |
Player X to 10
|X| | | | | | | | | |X|
Player X to 1 for score 1
|X|X| | | | | | | | |X|
Player X to 9
|X|X| | | | | | | |X|X|
Player X to 5 for score 1
|X|X| | | |X| | | |X|X|
Player X to 8
|X|X| | | |X| | |X|X|X|


negamaxIDS:
| | | | | | | | | | | |
Player X to 0 for score 1
|X| | | | | | | | | | |
Player X to 3
|X| | |X| | | | | | | |
Player X to 1 for score 1
|X|X| |X| | | | | | | |
Player X to 9
|X|X| |X| | | | | |X| |
Player X to 4 for score 1
|X|X| |X|X| | | | |X| |
Player X to 6
|X|X| |X|X| |X| | |X| |
Player X to 7 for score 1
|X|X| |X|X| |X|X| |X| |
Player X to 5
|X|X| |X|X|X|X|X| |X| |


negamaxIDSab:
| | | | | | | | | | | |
Player X to 0 for score 1
|X| | | | | | | | | | |
Player X to 10
|X| | | | | | | | | |X|
Player X to 1 for score 1
|X|X| | | | | | | | |X|
Player X to 3
|X|X| |X| | | | | | |X|
Player X to 4 for score 1
|X|X| |X|X| | | | | |X|
Player X to 8
|X|X| |X|X| | | |X| |X|
Player 

In above solution, there are times when player 2 is also winning. In Tic-Tac-Toe, even using player 2 as random, player one was winning as we was training player 1  but as in this game there is more possibility of any 3 consecutive 'X' in row, player 2 wins.